In [1]:
import glob, os, gc
import os.path
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from __future__ import division
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.decomposition import (PCA, RandomizedPCA)
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import pybedtools
import pysam
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/home/ubuntu/efs/SLAV_Data/" 
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
peaks_L1HS_bedgraph = "_peaks_L1HS_mapped.bedgraph"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
L1HS_bam = "-L1HS_mapped.bam"
bam = "-ready.bam"
igv = "-igv.xml"
bed = ".bed"
## rmask Paths 
L1HS = "/home/ubuntu/efs/SLAV_Data/rmask_L1HS_Final.bed"
L1PA2345 = "/home/ubuntu/efs/SLAV_Data/rmask_L1PA2345_Final.bed"
L1_Other = "/home/ubuntu/efs/SLAV_Data/rmask_L1_Other_Final.bed"
##IGV Template
IGV = "/home/ubuntu/efs/SLAV_Data/igv-template4.xml"


Bulk_5125_Cortex = "5125_cortex_BT_122_L3"
Bulk_5125_Hippocampus = "5125_hippo_BT_139_L3"
Bulk_5125_Liver = "5125_liver_BT_164_L3"

SC_5125_Cortex = ["5125_cortex_SC_125_L3","5125_cortex_SC_126_L3","5125_cortex_SC_127_L3","5125_cortex_SC_128_L3","5125_cortex_SC_129_L3","5125_cortex_SC_130_L3","5125_cortex_SC_131_L3","5125_cortex_SC_132_L3","5125_cortex_SC_133_L3","5125_cortex_SC_134_L3","5125_cortex_SC_135_L3","5125_cortex_SC_136_L3","5125_cortex_SC_138_L3"]


Data_Sets = []
Data_Sets.append([SC_5125_Cortex,Bulk_5125_Cortex,Bulk_5125_Liver])

In [ ]:
for dset in Data_Sets:
    tissue = dset[1]
    os.chdir(os.path.join(basepath, tissue))
    if not (glob.glob(os.path.join(basepath, tissue, "peaks.txt"))):
        print tissue
        name = os.path.join(basepath, tissue, tissue)
        bamfile = os.path.join(basepath, tissue, tissue+bam)
        #p1 = Popen(['/home/preed/homer/bin/makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['/home/preed/homer/bin/findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['/home/preed/homer/bin/pos2bed.pl', '-bed', os.path.join(basepath, tissue, "peaks.txt")])
        p4.wait()
    tissue = dset[2]
    os.chdir(os.path.join(basepath, tissue))
    if not (glob.glob(os.path.join(basepath, tissue, "peaks.txt"))):
        print tissue
        name = os.path.join(basepath, tissue, tissue)
        bamfile = os.path.join(basepath, tissue, tissue+bam)
        #p1 = Popen(['/home/preed/homer/bin/makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['/home/preed/homer/bin/findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['/home/preed/homer/bin/pos2bed.pl', '-bed', os.path.join(basepath, tissue, "peaks.txt")])
        p4.wait()                

In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))
        name = os.path.join(basepath, cell, cell)
        bamfile = os.path.join(basepath, cell, cell+bam)
        #p1 = Popen(['/home/preed/homer/bin/makeTagDirectory', '.', bamfile, '-format', 'sam', '-keepAll', '-single'])
        #p1.wait()
        p2a = Popen(['/home/preed/homer/bin/findPeaks', '.', '-o', 'peaks.txt', '-style', 'dnase', '-F', '0', '-L', '0', '-C', '0', '-tagThreshold', '5'])
        p2a.wait()
        p4 = Popen(['/home/preed/homer/bin/pos2bed.pl', '-bed', os.path.join(basepath, cell, "peaks.txt")])
        p4.wait()
        p5 = Popen(['/home/preed/homer/bin/mergePeaks', '-code', '-prefix', cell, os.path.join(basepath, cell, "peaks.txt"), os.path.join(basepath, dset[1], "peaks.txt"), os.path.join(basepath,  dset[2], "peaks.txt")])
        p5.wait()
        #myoutput = open(os.path.join(basepath, cell, cell + peaks_merged), 'w')
        #p5 = Popen(['/home/preed/homer/bin/mergePeaks', os.path.join(basepath, cell, "peaks.txt"), os.path.join(basepath, dset[1], "peaks.txt"), os.path.join(basepath,  dset[2], "peaks.txt")], stdout=myoutput)
        #p5.wait()
        myoutput = open(os.path.join(basepath, cell, cell + peaks_merged), 'w')
        p6 = Popen(['/home/preed/homer/bin/mergePeaks', os.path.join(basepath,  cell, cell+"_100"),os.path.join(basepath,  cell, cell+"_110"),os.path.join(basepath,  cell, cell+"_111")], stdout=myoutput)
        p6.wait()
        p7 = Popen(['/home/preed/homer/bin/pos2bed.pl', os.path.join(basepath,  cell, cell + peaks_merged), '-o', os.path.join(basepath,  cell, cell+peaks_merged_bed)])
        p7.wait() 

In [2]:
for dset in Data_Sets:
    for cell in dset[0]:
        print cell
        tree = ET.parse(IGV)
        root = tree.getroot()
        root[0][0].set('path', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[0][1].set('path', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[0][3].set('path', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain Path
        root[0][6].set('path', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        root[1][0].set('id', os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)) #L1HS bedgraph
        root[2][0].set('id', os.path.join(basepath,  cell, cell + bam)) #SC Path
        root[3][0].set('id', os.path.join(basepath,  dset[1], dset[1] + bam)) #Bulk Brain path
        root[4][0].set('id', os.path.join(basepath,  dset[2], dset[2] + bam)) #Bulk Fib Path
        tree.write(os.path.join(basepath,  cell, cell + igv))



#         myinput = open(os.path.join(basepath,  cell, cell + peaks_merged_bed))
#         myoutput = open(os.path.join(basepath,  cell, cell + peaks_correct_bed), 'w')
#         proc3 = Popen(['grep', '-E', '^(1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|X|Y)'], stdin=myinput, stdout=myoutput)
#         proc3.wait()    

#         myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
#         myoutput1 = os.path.join(basepath,  cell, cell + peakregions_sml)
#         myoutput2 = os.path.join(basepath,  cell, cell + peakregions_lrg)
#         with open(myoutput1, 'w') as outfile:
#             with open(myinput, 'r') as infile:
#                 data = infile.readlines()
#                 for string in data:
#                     line = string.split('\t')
#                     pos1 = int(line[1])
#                     pos2 = int(line[2])                  
#                     center = int((pos1 + pos2)/2)
#                     pad = 1000
#                     start = center - pad
#                     end = center + pad
#                     row = [line[0], str(start), str(end)]
#                     outfile.write('\t'.join(row) + '\n')
#         outfile.close()
#         infile.close()   
#         with open(myoutput2, 'w') as outfile:
#             with open(myinput, 'r') as infile:
#                 data = infile.readlines()
#                 for string in data:
#                     line = string.split('\t')
#                     pos1 = int(line[1])
#                     pos2 = int(line[2])                  
#                     center = int((pos1 + pos2)/2)
#                     pad = 10000
#                     newstart = center - pad
#                     newend = center + pad
#                     row = [line[0], str(newstart), str(newend)]
#                     outfile.write('\t'.join(row) + '\n')
#         outfile.close()
#         infile.close()
 
#         #make and define L1HS sub sam file here and L1HS read names list
        
               
#         sc_file = pysam.AlignmentFile(os.path.join(basepath,  cell, cell + bam), "rb")
#         bb_file = pysam.AlignmentFile(os.path.join(basepath,  dset[1], dset[1] + bam), "rb")
#         bf_file = pysam.AlignmentFile(os.path.join(basepath,  dset[2], dset[2] + bam), "rb")
        

#         myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
#         myoutput = os.path.join(basepath,  cell, cell + peaks_correct_data)
        
#         with open(myoutput, 'w') as outfile:
#             with open(myinput, 'r') as infile:
#                 data = infile.readlines()
#                 for region in data:
#                     sc_iter = sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
#                     bb_iter = bb_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
#                     bf_iter = bf_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2]))
#                     sc_i = 0
#                     bb_i = 0
#                     bf_i = 0
#                     for x in sc_iter: sc_i+=1
#                     for y in bb_iter: bb_i+=1
#                     for z in bf_iter: bf_i+=1
#                     sc_count = 1 if sc_i > 0 else 0
#                     bb_count = 1 if bb_i > 0 else 0 
#                     bf_count = 1 if bf_i > 0 else 0 
#                     row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2])]  
#                     outfile.write('\t'.join(row) +'\t'+str(sc_count)+str(bb_count)+str(bf_count)+'\n')
        
#         myinput = os.path.join(basepath,  cell, cell + peaks_correct_bed)
#         myoutput = open(os.path.join(basepath, cell, cell + L1HS_bam), 'w')
#         myoutput2 = os.path.join(basepath,  cell, cell + peaks_L1HS_bedgraph)
#         p1 = Popen(['samtools', 'view', '-hb', '-@', '1', '-L', L1HS, os.path.join(basepath, cell, cell + bam)], stdout=myoutput)
#         p1.wait()
#         p2 = Popen(['samtools', 'index', '-@', '1', os.path.join(basepath, cell, cell + L1HS_bam)])
#         p2.wait()
#         L1HS_file = pysam.AlignmentFile(os.path.join(basepath, cell, cell + L1HS_bam), 'rb')
#         L1HS_read_names =[]
#         for read in L1HS_file.fetch(): 
#             read_name = int(str(read).split('\t')[0].split('.')[1])
#             L1HS_read_names.append(read_name)
#         #print len(L1HS_read_names)    
#         with open(myoutput2, 'w') as outfile:
#             with open(myinput, 'r') as infile:
#                 data = infile.readlines()
#                 for region in data:
#                     read_names = []
#                     for read in sc_file.fetch(region.split('\t')[0], int(region.split('\t')[1]), int(region.split('\t')[2])):
#                         read_name = int(str(read).split('\t')[0].split('.')[1])
#                         read_names.append(read_name)
#                     a = len(read_names)    
#                     L1HS_overlap = set(read_names) & set(L1HS_read_names)
#                     b = len(L1HS_overlap)
#                     percent = int((b/a)*100)
#                     row = [str(region.strip().split('\t')[0]), str(region.strip().split('\t')[1]), str(region.strip().split('\t')[2]), str(percent)]  
#                     outfile.write('\t'.join(row)+'\n')

#         filelist =[os.path.join(basepath,  cell, "peaks.bed"),os.path.join(basepath,  dset[1], "peaks.bed"),os.path.join(basepath,  dset[2], "peaks.bed"),L1HS,L1PA2345,L1_Other]
#         a = pybedtools.BedTool(os.path.join(basepath, cell, cell + peaks_correct_bed)) ##mergedpeaks2 overlap with loci window or with peak location???
#         count = 0
#         for fname in filelist:
#             b = pybedtools.BedTool(fname)
#             a_and_b = a.intersect(b, c=True)
#             myoutput = os.path.join(basepath,  cell, cell + overlap + str(count))
#             count +=1
#             a_and_b.saveas(myoutput)
#             myinput = open(myoutput)
#             newoutput = open(myoutput+"_binary", 'w')
#             #overlap_append
#             awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($7 ~ "^[1-9]*$") $7 = "1"; else $7 = $7; }; 1"""
#             proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
#             proc.wait()
#             newoutput.flush()

#         a_sml = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_sml)) ##mergedpeaks2 overlap with loci window or with peak location???
#         count = 0
#         for fname in filelist:
#             b = pybedtools.BedTool(fname)
#             a_and_b = a_sml.intersect(b, c=True)
#             myoutput = os.path.join(basepath,  cell, cell + overlap_sml + str(count))
#             count +=1
#             a_and_b.saveas(myoutput)
#             myinput = open(myoutput)
#             newoutput = open(myoutput+"_binary", 'w')
#             #overlap_append
#             awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
#             proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
#             proc.wait()
#             newoutput.flush()

#         a_lrg = pybedtools.BedTool(os.path.join(basepath,  cell, cell + peakregions_lrg)) ##mergedpeaks2 overlap with loci window or with peak location???
#         count = 0
#         for fname in filelist:
#             b = pybedtools.BedTool(fname)
#             a_and_b = a_lrg.intersect(b, c=True)
#             myoutput = os.path.join(basepath,  cell, cell + overlap_lrg + str(count))
#             count +=1
#             a_and_b.saveas(myoutput)
#             myinput = open(myoutput)
#             newoutput = open(myoutput+"_binary", 'w')
#             #overlap_append
#             awk_cmd = r"""BEGIN { OFS = "\t"; }; { if ($4 >= 2) $4 = "2"; else $4 = $4; }; 1"""
#             proc = Popen(['awk', awk_cmd], stdin=myinput, stdout=newoutput)  
#             proc.wait()
#             newoutput.flush()        

#         myinput_sml = os.path.join(basepath,  cell, cell + peakregions_sml)
#         myoutput_sml = os.path.join(basepath,  cell, cell + loci_sml)
#         with open(myoutput_sml, 'w') as outfile:
#             with open(myinput_sml, 'r') as infile:
#                 data = infile.readlines()
#                 for region in data:
#                     row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
#                     outfile.write("".join(row)+'\n')

#         myinput_lrg = os.path.join(basepath,  cell, cell + peakregions_lrg)
#         myoutput_lrg = os.path.join(basepath,  cell, cell + loci_lrg)
#         with open(myoutput_lrg, 'w') as outfile:
#             with open(myinput_lrg, 'r') as infile:
#                 data = infile.readlines()
#                 for region in data:
#                     row = [str(region.strip().split('\t')[0]),":",str(region.strip().split('\t')[1]),"-",str(region.strip().split('\t')[2])]  
#                     outfile.write("".join(row)+'\n')    

#         Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_sml), os.path.join(basepath,  cell, cell + ".split_loci_sml_")]).wait()
#         Popen(['split', '-l', '100', '-d', os.path.join(basepath,  cell, cell + loci_lrg), os.path.join(basepath,  cell, cell + ".split_loci_lrg_")]).wait()

5125_cortex_SC_125_L3
5125_cortex_SC_126_L3
5125_cortex_SC_127_L3
5125_cortex_SC_128_L3
5125_cortex_SC_129_L3
5125_cortex_SC_130_L3
5125_cortex_SC_131_L3
5125_cortex_SC_132_L3
5125_cortex_SC_133_L3
5125_cortex_SC_134_L3
5125_cortex_SC_135_L3
5125_cortex_SC_136_L3
5125_cortex_SC_138_L3


In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))    
        locifile = os.path.join(basepath, cell, cell + loci_sml)
        worklist = glob.glob("*.split_loci_sml_*")
        batchsize = 10
        print len(worklist)
        for i in xrange(0, len(worklist), batchsize):
            batch = worklist[i:i+batchsize]
            print i
            index = 1
            procs = []
            for file in batch:
                print file
                with open(os.path.join(basepath, cell, file)) as f0:
                    first = f0.readline()# Read the first line.
                    for last in f0: pass
                    firstpic = cell+"_sml"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                    lastpic = cell+"_sml"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                    if not (glob.glob(os.path.join(basepath, cell, firstpic)) or glob.glob(os.path.join(basepath, cell, lastpic))): 
                        p = Popen(['igv_plotter', '-o', cell+"_sml_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/home/ubuntu/efs/SLAV_Data/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, cell, cell + igv)])
                        procs.append(p)
            for pp in procs:
                pp.wait()
                #wait_timeout(pp,300)

5125_cortex_SC_125_L3
162
0
5125_cortex_SC_125_L3.split_loci_sml_9002
5125_cortex_SC_125_L3.split_loci_sml_38
5125_cortex_SC_125_L3.split_loci_sml_74
5125_cortex_SC_125_L3.split_loci_sml_9020
5125_cortex_SC_125_L3.split_loci_sml_56
5125_cortex_SC_125_L3.split_loci_sml_9011
5125_cortex_SC_125_L3.split_loci_sml_47
5125_cortex_SC_125_L3.split_loci_sml_83
5125_cortex_SC_125_L3.split_loci_sml_65
5125_cortex_SC_125_L3.split_loci_sml_29
10
5125_cortex_SC_125_L3.split_loci_sml_9046
5125_cortex_SC_125_L3.split_loci_sml_03
5125_cortex_SC_125_L3.split_loci_sml_9028
5125_cortex_SC_125_L3.split_loci_sml_9064
5125_cortex_SC_125_L3.split_loci_sml_21
5125_cortex_SC_125_L3.split_loci_sml_9019
5125_cortex_SC_125_L3.split_loci_sml_9055
5125_cortex_SC_125_L3.split_loci_sml_12
5125_cortex_SC_125_L3.split_loci_sml_9037
5125_cortex_SC_125_L3.split_loci_sml_30
20
5125_cortex_SC_125_L3.split_loci_sml_9006
5125_cortex_SC_125_L3.split_loci_sml_9042
5125_cortex_SC_125_L3.split_loci_sml_78
5125_cortex_SC_125_L3.sp

5125_cortex_SC_126_L3.split_loci_sml_08
5125_cortex_SC_126_L3.split_loci_sml_44
40
5125_cortex_SC_126_L3.split_loci_sml_26
5125_cortex_SC_126_L3.split_loci_sml_62
5125_cortex_SC_126_L3.split_loci_sml_65
5125_cortex_SC_126_L3.split_loci_sml_29
5125_cortex_SC_126_L3.split_loci_sml_47
5125_cortex_SC_126_L3.split_loci_sml_74
5125_cortex_SC_126_L3.split_loci_sml_38
5125_cortex_SC_126_L3.split_loci_sml_56
5125_cortex_SC_126_L3.split_loci_sml_30
5125_cortex_SC_126_L3.split_loci_sml_12
50
5125_cortex_SC_126_L3.split_loci_sml_03
5125_cortex_SC_126_L3.split_loci_sml_21
5125_cortex_SC_126_L3.split_loci_sml_69
5125_cortex_SC_126_L3.split_loci_sml_78
5125_cortex_SC_126_L3.split_loci_sml_70
5125_cortex_SC_126_L3.split_loci_sml_34
5125_cortex_SC_126_L3.split_loci_sml_16
5125_cortex_SC_126_L3.split_loci_sml_52
5125_cortex_SC_126_L3.split_loci_sml_07
5125_cortex_SC_126_L3.split_loci_sml_43
60
5125_cortex_SC_126_L3.split_loci_sml_25
5125_cortex_SC_126_L3.split_loci_sml_61
5125_cortex_SC_126_L3.split_loc

In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, diry, cell))    
        locifile = os.path.join(basepath, diry, cell, cell + loci_lrg)
        worklist = glob.glob("*.split_loci_lrg_*")
        batchsize = 10
        print len(worklist)
        for i in xrange(0, len(worklist), batchsize):
            batch = worklist[i:i+batchsize]
            print i
            index = 1
            procs = []
            for file in batch:
                print file
                with open(os.path.join(basepath, diry, cell, file)) as f0:
                    first = f0.readline()# Read the first line.
                    for last in f0: pass
                    firstpic = cell+"_lrg"+"*"+first.strip().split(':')[0]+"_"+first.strip().split(':')[1].split('-')[0]+"_"+first.strip().split(':')[1].split('-')[1]+".png"
                    lastpic = cell+"_lrg"+"*"+last.strip().split(':')[0]+"_"+last.strip().split(':')[1].split('-')[0]+"_"+last.strip().split(':')[1].split('-')[1]+".png"
                    if not (glob.glob(os.path.join(basepath, diry, cell, firstpic)) or glob.glob(os.path.join(basepath, diry, cell, lastpic))): 
                        p = Popen(['igv_plotter', '-o', cell+"_lrg_", '-L', file, '-v', '--max-panel-height', '1000', '--igv-jar-path', '/raid/LOG-G/SEQ_ARCHIVE/Patrick_Reed/BSMN/IGV_2.4-rc6/igv.jar', '-m', '6G', '-g', 'hg19', os.path.join(basepath, diry, cell, cell + igv)])
                        procs.append(p)
            for pp in procs:
                pp.wait()
                #wait_timeout(pp,300)

In [ ]:
for dset in Data_Sets:    
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, diry, cell))
        for file in glob.glob("*s*__*.png"):
            newfile = re.sub("_s\d+__", "-", file)
            shutil.move(file, newfile)     
        for file in glob.glob("*.png"):
            img = Image.open(file)
            width = img.size[0]
            height = img.size[1]
            img2 = img.crop((70,130,width,height)).resize((200,500))
            path = os.path.splitext(file)[0]
            basename = os.path.basename(path)
            outfile1 = basename + "_crp.png"
            img2.save(outfile1)
            #os.remove(file)

        mergedpeak_data = os.path.join(basepath, diry, cell, cell + peaks_correct_data)
        regions_sml = os.path.join(basepath, diry, cell, cell + peakregions_sml)
        regions_lrg = os.path.join(basepath, diry, cell, cell + peakregions_lrg)

        count=1
        with open(mergedpeak_data) as r0,open(regions_sml) as r_sml,open(regions_lrg) as r_lrg:
            Files= {}
            for rr0,rr_sml,rr_lrg in zip(r0,r_sml,r_lrg):
                line = rr0.strip().split('\t')[0]+"\t"+rr0.strip().split('\t')[1]+"\t"+rr0.strip().split('\t')[2]+"\t"+cell+"_sml-"+rr_sml.strip().split('\t')[0]+"_"+rr_sml.strip().split('\t')[1]+"_"+rr_sml.strip().split('\t')[2]+"_crp.png"+"\t"+cell+"_lrg-"+rr_lrg.strip().split('\t')[0]+"_"+rr_lrg.strip().split('\t')[1]+"_"+rr_lrg.strip().split('\t')[2]+"_crp.png"+"\t"+rr0.strip().split('\t')[3]
                Files[str(count)] = line
                count+=1

        a = os.path.join(basepath, diry, cell, cell+"_overlap_0_binary")
        b = os.path.join(basepath, diry, cell, cell+"_overlap_1_binary")
        c = os.path.join(basepath, diry, cell, cell+"_overlap_2_binary")
        d = os.path.join(basepath, diry, cell, cell+"_overlap_3_binary")
        e = os.path.join(basepath, diry, cell, cell+"_overlap_4_binary")
        f = os.path.join(basepath, diry, cell, cell+"_overlap_5_binary")
        count=1
        with open(a) as f1,open(b) as f2,open(c) as f3,open(d) as f4,open(e) as f5,open(f) as f6:
            Peaks = {}
            for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
                line = aa.strip().split('\t')[6]+bb.strip().split('\t')[6]+cc.strip().split('\t')[6]+"\t"+dd.strip().split('\t')[6]+ee.strip().split('\t')[6]+ff.strip().split('\t')[6]
                Peaks[str(count)] = line
                count+=1

        a_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_0_binary")
        b_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_1_binary")
        c_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_2_binary")
        d_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_3_binary")
        e_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_4_binary")
        f_sml = os.path.join(basepath, diry, cell, cell+"_overlap_sml_5_binary")
        count=1
        with open(a_sml) as f1,open(b_sml) as f2,open(c_sml) as f3,open(d_sml) as f4,open(e_sml) as f5,open(f_sml) as f6:
            Small = {}
            for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
                line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
                Small[str(count)] = line
                count+=1

        a_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_0_binary")
        b_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_1_binary")
        c_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_2_binary")
        d_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_3_binary")
        e_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_4_binary")
        f_lrg = os.path.join(basepath, diry, cell, cell+"_overlap_lrg_5_binary")
        count=1
        with open(a_lrg) as f1,open(b_lrg) as f2,open(c_lrg) as f3,open(d_lrg) as f4,open(e_lrg) as f5,open(f_lrg) as f6:
            Large = {}
            for aa,bb,cc,dd,ee,ff in zip(f1,f2,f3,f4,f5,f6):
                line = aa.strip().split('\t')[3]+bb.strip().split('\t')[3]+cc.strip().split('\t')[3]+"\t"+dd.strip().split('\t')[3]+ee.strip().split('\t')[3]+ff.strip().split('\t')[3]
                Large[str(count)] = line
                count+=1

        with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"),"w") as f8:
            for key in Files:
                encoding = Files[key]+":"+Peaks[key].strip().split('\t')[0]+":"+Large[key].strip().split('\t')[0]+":"+Peaks[key].strip().split('\t')[1]+"\n"
            #for (Fi, Fj),(Pi,Pj),(Si,Sj),(Li,Lj) in zip(Files.items(),Peaks.items(),Small.items(),Large.items()):
                #encoding = Fj+":"+Pj.strip().split('\t')[0]+":"+Sj.strip().split('\t')[0]+":"+Lj.strip().split('\t')[0]+":"+Pj.strip().split('\t')[1]+":"+Sj.strip().split('\t')[1]+":"+Lj.strip().split('\t')[1]+"\n"
                f8.write(encoding)

    #    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
    #        for line in csv.reader(f, delimiter="\t"):
    #            if os.path.isfile(line[3]):
    #                filename = line[3]
    #                dst = line[5]
    #                if not os.path.exists(dst):
    #                    os.makedirs(os.path.join(basepath, diry, cell, dst))
    #                shutil.move(filename, os.path.join(basepath, diry, cell, dst))

    #    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
    #        for line in csv.reader(f, delimiter="\t"):
    #            if os.path.isfile(line[4]):
    #                filename = line[4]
    #                dst = line[5]
    #                if not os.path.exists(dst):
    #                    os.makedirs(os.path.join(basepath, diry, cell, dst))
    #                shutil.move(filename, os.path.join(basepath, diry, cell, dst))                
        

In [ ]:
print file

In [ ]:
filelist_sml = []
metadata_sml = []
#metadata2_sml = []

for cell in SingleCell_MDA:
    print cell
    #shutil.copy(os.path.join(basepath, "final", cell, cell+"_Input_metadata.txt"),os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"))
    os.chdir(os.path.join(basepath, diry, cell))
## Load images and meta data from meta files
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
        for line in csv.reader(f, delimiter="\t"):
            if os.path.isfile(line[3]):
                readclass = line[5].split(":")[0]
                peakclass_sml = line[5].split(":")[1]               
                L1_class = line[5].split(":")[3]
                #print os.path.join(basepath, diry, cell, line[3])
                #if peakclass_sml == readclass:
                class_sml = [readclass,L1_class]
                filelist_sml.append(os.path.join(basepath, diry, cell, line[3])) 
                metadata_sml.append("".join(class_sml))
                #metadata2_sml.append("".join(L1_class))
                
os.chdir(os.path.join(basepath))
X_sml = np.array([np.array(Image.open(fname)) for fname in filelist_sml])
Y_sml = np.array(metadata_sml).astype(str)
print X_sml.shape
print Y_sml.size
print len(np.unique(Y_sml))
n_classes_sml = Y_sml.size
n_samples_sml = len(X_sml)
np.savez("MDA_Cells_sml", X_sml=X_sml, Y_sml=Y_sml)

#X2 = X.reshape((n_sam,-1))
#print X2.shape
#X = None



In [ ]:
print Y_sml

In [ ]:
filelist_lrg = []
metadata_lrg = []

for cell in SingleCell_MDA:
    print cell
    #shutil.copy(os.path.join(basepath, "final", cell, cell+"_Input_metadata.txt"),os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt"))
    os.chdir(os.path.join(basepath, diry, cell))
## Load images and meta data from meta files
    
    with open(os.path.join(basepath, diry, cell, cell+"_Input_metadata.txt")) as f:
        for line in csv.reader(f, delimiter="\t"):
            if os.path.isfile(line[4]):
                peakclass_lrg = line[5].split(":")[2]               
                L1_class = line[5].split(":")[3]
                #print os.path.join(basepath, diry, cell, line[3])
                filelist_lrg.append(os.path.join(basepath, diry, cell, line[4]))
                class_lrg = [peakclass_lrg,L1_class]
                metadata_lrg.append("".join(Class_lrg))
                
os.chdir(os.path.join(basepath))
X_lrg = np.array([np.array(Image.open(fname)) for fname in filelist_lrg])
Y_lrg = np.array(metadata_lrg).astype(str)
print X_lrg.shape
print Y_lrg.size
print len(np.unique(Y_lrg))
n_classes_lrg = Y_lrg.size
n_samples_lrg = len(X_lrg)
np.savez("MDA_Cells_lrg", X_lrg=X_lrg, Y_lrg=Y_lrg)

#X2 = X.reshape((n_sam,-1))
#print X2.shape
#X = None
#np.savez("All_Cells_sml", X2=X2, Y=Y)



In [ ]:
####Test Basic Convolution Model

import random
np.random.seed(123)  # for reproducibility
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

config = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32, allow_soft_placement=True, device_count = {'CPU': 32})
session = tf.Session(config=config)
K.set_session(session)

# 6. Preprocess class labels
Ydict_1 = {}
Ydict_2 = {}
Y_unique = np.unique(Y_sml)
print Y_unique
i=0
for y_u in Y_unique:
    Ydict_1[i] = y_u
    Ydict_2[y_u] = i
    i+=1
print Ydict_2
trainsize = 20000
testsize = 10000
train_test = random.sample(range(0, n_samples_sml),trainsize+testsize)
train = train_test[0:trainsize]
test = train_test[trainsize:trainsize+testsize]
X_train = []
X_test = []
X_train = X_sml[train,:]
X_test = X_sml[test,:]
Ytr = [Ydict_2[y_train] for y_train in Y_sml[train]]
Yte = [Ydict_2[y_test] for y_test in Y_sml[test]]
Y_train = np_utils.to_categorical(Ytr)
Y_test = np_utils.to_categorical(Yte)

num_categories = len(np.unique(Y_sml))
print num_categories
# 7. Define model architecture
model = Sequential() 
model.add(Conv2D(32, (3,3), activation='relu', data_format="channels_first", input_shape=(3,500,200)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

# 8. Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

X_train = X_train.reshape(X_train.shape[0], 3, 500, 200)
X_test = X_test.reshape(X_test.shape[0], 3, 500, 200)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 9. Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

score = model.evaluate(X_test, Y_test, verbose=1)

score

classes = model.predict(X_test, batch_size=128)

classes

X_test.shape

classes.shape

classes[1,]

In [ ]:
####Test Basic Convolution Model

import random
np.random.seed(123)  # for reproducibility
import tensorflow as tf
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

config = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32, allow_soft_placement=True, device_count = {'CPU': 32})
session = tf.Session(config=config)
K.set_session(session)

# 6. Preprocess class labels
Ydict_1 = {}
Ydict_2 = {}
Y_unique = np.unique(Y1_1)
print Y_unique
i=0
for y_u in Y_unique:
    Ydict_1[i] = y_u
    Ydict_2[y_u] = i
    i+=1
print Ydict_2
trainsize = 20000
testsize = 10000
train_test = random.sample(range(0, n_sam),trainsize+testsize)
train = train_test[0:trainsize]
test = train_test[trainsize:trainsize+testsize]
X_train = []
X_test = []
X_train = X1[train,:]
X_test = X1[test,:]
Ytr = [Ydict_2[y_train] for y_train in Y1_1[train]]
Yte = [Ydict_2[y_test] for y_test in Y1_1[test]]
Y_train = np_utils.to_categorical(Ytr)
Y_test = np_utils.to_categorical(Yte)

num_categories = len(np.unique(Y1_1))
print num_categories
# 7. Define model architecture
model = InceptionV3(weights='None', include_top=False)

# 8. Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

X_train = X_train.reshape(X_train.shape[0], 3, 500, 200)
X_test = X_test.reshape(X_test.shape[0], 3, 500, 200)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 9. Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1)

score = model.evaluate(X_test, Y_test, verbose=1)

score

classes = model.predict(X_test, batch_size=128)

classes

X_test.shape

classes.shape

classes[1,]

In [ ]:
print score

In [ ]:
print classes.shape